<a href="https://colab.research.google.com/github/DEKDEGUE-Hajar/Extension-Innovante-Pour-La-Compression-Decompression-de-Fichier-Audio/blob/main/Extension_Innovante_Pour_La_Compression_Decompression_de_Fichier_Audio_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 <h1 style="text-align: justify; font-weight: bold;>hzllo</h1>

<h1><font color='red'><text-align: justify>
Création d'Une Extension Innovante Pour La Compression/Decompression de Fichier Audio
</h1>


**Objectif:**  Concevoir un format innovant de **compression** et **décompression** audio, capable de réduire efficacement la taille des fichiers audio courants (WAV, MP3, OGG, etc.) tout en permettant de retrouver l’audio original après décompression. Ce nouveau format, nommé **IRM** (Informatique, Réseaux et Multimédia), en référence à notre filière de Licence, a pour but de minimiser la taille des fichiers lors de la compression tout en garantissant une qualité audio optimale. L'accent est mis sur la réduction significative de la taille des fichiers sans compromettre la qualité, assurant ainsi un format efficace pour le stockage et le partage des fichiers multimédias.

<h2><font color='blue'><text-align: justify>
0.Importation des bibliothèques nécessaires
</h2>

In [ ]:
import numpy as np
from scipy import signal
import sounddevice as sd
from tkinter import filedialog
import tkinter as tk
import wave
import pywt
import bitarray
import os
import pydub
import struct
import pickle
from pydub import AudioSegment


<h2><font color='blue'><text-align: justify>
1.Conversion binaire
</h2>


La fonction (from_dec_to_bin) prend une séquence d’entiers décimaux et renvoie une chaîne binaire représentant ces entiers.

In [ ]:
def from_dec_to_bin(sequence):
    binary="".join([format(int(elem),"08b") for elem in sequence])
    return binary

<h2><font color='blue'><text-align: justify>
2. Conversion Décimale
</h2>


La fonction (from_bin_to_decimal) prend une chaîne binaire comme entrée, la décode en une séquence d’entiers décimaux et retourne la séquence résultante.

In [ ]:
def from_bin_to_decimal(decode_RLE):
    seq=[int(decode_RLE[i:8+i], 2) for i in range(0,len(decode_RLE),8)]
    return seq

<h2><font color='blue'><text-align: justify>
3. LZW
</h2>

**LZW (Lempel-Ziv-Welch)** est un algorithme de compression sans perte, largement utilisé pour réduire la taille des fichiers en remplaçant les séquences de caractères répétées par des codes courts. Cet algorithme a été développé par Abraham Lempel, Jacob Ziv, et Terry Welch dans les années 1980.

### Fonctionnement :
1. **Table de symboles :** L'algorithme commence par créer une table qui associe chaque symbole unique du fichier à un code. Par défaut, chaque caractère individuel est initialisé dans cette table.
  
2. **Lecture du fichier :** LZW lit les données du fichier d'entrée et construit progressivement une chaîne à partir des symboles déjà trouvés.

3. **Compression :** Lorsque l'algorithme rencontre une nouvelle séquence de caractères qui n'est pas encore dans la table, il l'ajoute à la table et émet le code correspondant à la plus longue séquence trouvée précédemment.

4. **Sortie :** Le résultat est un fichier compressé qui contient uniquement des codes, ce qui réduit significativement la taille du fichier par rapport à l'original.

### Avantages :
- **Efficacité :** LZW est particulièrement efficace pour les données contenant des motifs répétitifs, ce qui en fait un choix populaire pour des formats de fichiers comme **GIF** et **TIFF**.
  
- **Sans perte :** Étant un algorithme de compression sans perte, il garantit que les données originales peuvent être entièrement restaurées après décompression, sans aucune perte de qualité.

### Inconvénients :
- **Performance variable :** La performance de l'algorithme peut diminuer sur des données qui ne contiennent pas beaucoup de répétitions ou de motifs, rendant son efficacité variable selon le type de fichier.

### Applications :
LZW est utilisé dans plusieurs formats de fichiers et protocoles, y compris :
- **GIF :** Pour la compression d'images.
- **TIFF :** Pour la compression d'images de haute qualité.
- **PDF :** Pour la compression de documents.

En résumé, LZW est un algorithme de compression puissant, efficace et largement utilisé, permettant de réduire la taille des fichiers tout en préservant l'intégrité des données.


In [ ]:
def compress(data):
    dictionary = {"0":1, "1":2}
    current_code = 3
    result = []
    sequence = ""
    for symbol in data:
        new_sequence = sequence + symbol
        if new_sequence in dictionary:
            sequence = new_sequence
        else:
            result.append(dictionary[sequence])
            dictionary[new_sequence] = current_code
            current_code += 1
            sequence = symbol
    if sequence:
        result.append(dictionary[sequence])
    return result
def decompress(data):
    dictionary = {1:"0", 2:"1"}
    current_code = 3
    sequence = dictionary[data[0]]
    result = sequence
    for code in data[1:]:
        if code in dictionary:
            entry = dictionary[code]
        elif code == current_code:
            entry = sequence + sequence[0]
        result += entry
        dictionary[current_code] = sequence + entry[0]
        current_code += 1
        sequence = entry
    return result

###### La conversion de n’importe quelle extension à l'extension wav

In [ ]:
def Open_file():
    fenetre=tk.Tk()
    # demander à l'utilisateur de sélctionner un fichier audio
    file_path = filedialog.askopenfilename()
    fenetre.destroy()


    # Charger un fichier audio à l’aide de PyDub
    sound = pydub.AudioSegment.from_file(file_path)

   # extraire l'extension du fichier
    file_ext = os.path.splitext(file_path)[1]

   # convertir au format wav et enregistrer
    if file_ext != ".wav":
        new_file_path = os.path.splitext(file_path)[0] + ".wav"
        sound.export(new_file_path, format="wav")
        file_path=new_file_path
    return file_path


###### La conversion du format wav à n’importe quelle extension

In [ ]:
def convert_audio(file_path, new_format):

    # Ouvrir le fichier audio avec pydub
    sound = AudioSegment.from_wav(file_path)

    # Obtenir le nom du fichier d'origine et le nouveau nom de fichier avec l'extension de format donnée
    file_name = file_path.split('.')[0]
    new_file_name = file_name + '.' + new_format

    # Exporter le fichier audio vers le nouveau format
    sound.export(new_file_name, format=new_format)

    # Afficher un message de confirmation
    print(f"Fichier {file_path} converti en {new_file_name}")
convert_audio(Open_file(), "mp3")

Fichier D:/AUDIO/AEROHeli_Helicoptere 1 (ID 0263)_LS.wav converti en D:/AUDIO/AEROHeli_Helicoptere 1 (ID 0263)_LS.mp3


Cette fonction prend les données audio sous forme de liste d’entiers, regroupe chaque échantillon dans une valeur entière de 4 octets et stocke les métadonnées et les données audio sous forme de fichier binaire à l’aide de l’extension IRM.

In [ ]:
def convert_to_binary(n_channels,sample_rate,sampwidth,data_audio,coeffs_max,coeffs_lenght):

    # Créer un dictionnaire contenant des informations de métadonnées sur le fichier audio
    header = {
        "extention": "irm",
        "n_channels": n_channels,
        "sample_rate": sample_rate,
        "sampwidth": sampwidth,
        "length": len(data_audio),
        "coeffs_max": coeffs_max,
        "coeffs_lenght": coeffs_lenght
    }

    # Créer un objet bytearray pour stocker les données audio sous forme de flux d’octets
    data = bytearray()

    # Boucle à travers chaque échantillon dans les données audio
    for n in data_audio:
        # Compressez l’exemple dans une valeur entière de 4 octets et ajoutez-le au bytearray de données
        data.extend(struct.pack('i', n))

    # Ouvrez un fichier binaire appelé « compressed_file.irm » en mode écriture
    # et écrivez le dictionnaire de métadonnées et les données audio dans le fichier
    with open("compressed_file.irm", "wb") as f:
        pickle.dump(header, f)
        f.write(data)

Ce code lit un fichier binaire qui a été précédemment compressé à l’aide de la fonction convert_to_binary()

In [ ]:
def read_binary_file():
    # Ouvrez le fichier binaire pour la lecture
    with open("compressed_file.irm", "rb") as f:
        # Charger les donnés d’en-tête du fichier à l’aide de pickle
        header = pickle.load(f)
        # Calculer le nombre total d’octets dans les données de coefficient compressées
        num_bytes = header['length'] * 4
        # Lire les données de coefficient compressées à partir du fichier
        binary_data = f.read(num_bytes)
        data_audio = []
        for i in range(header['length']):
            # Décompressez 4 octets des données binaires et interprétez-les comme un entier de 32 bits
            # ajoutez-le à la liste des données audio
            data_audio.append(struct.unpack('i', binary_data[i*4:i*4+4])[0])
        # Extraire les valeurs de coefficient maximum et les longueurs des tableaux de coefficients compressés de l’en-tête
        coeffs_max = header['coeffs_max']
        coeffs_lenght = header['coeffs_lenght']
    # Renvoyer les métadonnées du fichier audio (nombre de canaux, fréquence d’échantillonnage, largeur d’échantillon), les valeurs de coefficient maximales
    # les longueurs des tableaux de coefficients compressés et les données de coefficients compressées sous forme de liste d’entiers
    return header['n_channels'],header['sample_rate'], header['sampwidth'],coeffs_max,coeffs_lenght,data_audio


<h2><font color='blue'>4. La transformée en ondelettes DWT</font></h2>


**DWT** (Transformée en Ondelette Discrète) est une technique de traitement du signal qui consiste à transformer un signal en un ensemble de composantes fréquentielles appelées ondelettes. Elle a été développée comme une alternative à la Transformée de Fourier à Court Terme (STFT) pour l'analyse temps-fréquence.

La compression basée sur la DWT est largement utilisée pour compresser les fichiers audio. Cette méthode décompose le signal audio en plusieurs composantes fréquentielles, appelées sous-bandes, à l'aide de la transformée en ondelettes discrète. Le principe clé de cette technique est d'identifier et de supprimer les fréquences les moins importantes, qui contiennent peu d'informat


**Pour effectuer la compression, nous suivrons les étapes suivantes:**

<h6><font color='blue'>1. Le choix de la fonction ondelette mère :</font></h6>
Dans ce cas, nous avons choisi <b>'db4'</b>, l’ondelette utilisée est une ondelette Daubechies 4 (db4).

<h6><font color='green'>2. La décomposition des ondelettes :</font></h6>
<font color='green'>level=5</font> : il s’agit du nombre de niveaux de décomposition que nous allons calculer. Dans ce cas, le signal est décomposé en <b>5 niveaux</b>.

<h6><font color='orange'>3. La troncature des coefficients :</font></h6>
- Estimer l’écart type du bruit présent dans les coefficients de détail au niveau le plus élevé de la décomposition en ondelettes.
- Définir un seuil pour la méthode <b>VisuShrink</b> en fonction de l’écart type estimé du bruit et de la longueur du signal.
- Appliquer un <b>seuillage souple</b> aux coefficients ondelettes à l’aide du seuil calculé.

<h6><font color='purple'>4. Encodage des coefficients d'ondelettes quantifiés en bits :</font></h6>
Ce schéma de codage utilise une représentation de longueur fixe pour chaque coefficient, chaque coefficient étant représenté à l’aide de <b>8 bits</b>. Le signe du coefficient est représenté par le bit le plus significatif, les coefficients positifs ayant un <b>0</b> dans le bit le plus significatif et les coefficients négatifs ayant un <b>1</b>.

<h6><font color='red'>5. Compression LZW :</font></h6>
La dernière étape consiste à appliquer la méthode de compression <b>LZW</b>.


<h2><font color='green'>4.1 La Compression</font></h2>


In [ ]:
def compress_to_irm(filename):
    # Lire le fichier audio
    file = wave.open(filename, mode='rb')
    sampwidth=file.getsampwidth()*8
    n_channels=file.getnchannels()
    sample_rate=file.getframerate()
    signal = np.frombuffer(file.readframes(-1), dtype='int16')
    # Appliquer la transformation DWT
    coeffs = pywt.wavedec(signal, 'db4', level=5)
    # Estimer le niveau sonore
    sigma = np.median(np.abs(coeffs[-1])) / 0.6745
    # Seuillage à l’aide de la méthode VisuShrink
    threshold = sigma * np.sqrt(2 * np.log(len(signal)))
    coeffs = [pywt.threshold(i, threshold, 'soft') for i in coeffs]

    # Compression des coefficients d'ondelettes
    ba =bitarray.bitarray(endian='little')
    ba_lzw= bitarray.bitarray(endian='little')
    coeffs_max=[]
    coeffs_lenghts=[]
    for c in coeffs:
        # Conversion des coefficients d'ondelettes en tableau NumPy
        c = np.asarray(c)
        # Calcul de la valeur maximale des coefficients d'ondelettes
        max_value = max(abs(c))

        coeffs_max.append(max_value)
        coeffs_lenghts.append(len(c))

        # Normalisation des coefficients d'ondelettes
        c_norm = c / (max_value + 1)
        # Quantification des coefficients d'ondelettes normalisés
        c_quant = np.round(c_norm * 127)

        # Encodage des coefficients d'ondelettes quantifiés en bits
        for b in c_quant:
            ba.extend("{0:b}".format(int(b) + 128).zfill(8))

    code_lzw=compress(''.join([str(x) for x in ba]))
    code_lzw=[int(x) for x in code_lzw]
    convert_to_binary(n_channels,sample_rate,sampwidth,code_lzw,coeffs_max,coeffs_lenghts)
    # Affichage des informations de compression
    original_size = os.path.getsize(filename)
    compressed_size = os.path.getsize("compressed_file.irm")
    compression_ratio = float(original_size) / compressed_size
    print("Original size: {} bytes".format(original_size))
    print("Compressed size: {} bytes".format(compressed_size))
    print("Compression ratio: {:.2f}:1".format(compression_ratio))

compress_to_irm(Open_file())

Original size: 1103820 bytes
Compressed size: 94892 bytes
Compression ratio: 11.63:1


<h3><font color='green'>4.1 Décompression</font></h3>


**Pour effectuer la décompression, nous suivons le étapes suivantes:**

1. Décodage LZW

2. Effectuer la quantification inverse

3. Appliquer la transformée en ondelettes discrètes inverses

4. Réaliser un décalage de niveau inverse et une mise à l’échelle sur le signal reconstruit pour obtenir le signal d'origine.


In [ ]:
def decompress_irm1():
    # lire les coefficients compressés
    n_channels,sample_rate,sampwidth,coeffs_max,coeffs_lenght,data_audio=read_binary_file()
    ba=decompress(data_audio)
    # Decoder les coefficients compressés
    coeffs_decoded = []
    start = 0
    for i in range(len(coeffs_lenght)):
        # extraire la longueur des tableaux de coefficients
        length = coeffs_lenght[i]

        # Decodage des tableaux de coefficients
        c_bits = ba[start:start+length*8]
        c_bits=[str(x) for x in c_bits]
        c_int = np.array(from_bin_to_decimal("".join(c_bits)))-128

        # Redimensionner et quantifier inversement le tableau de coefficients
        c_norm = c_int / 127

        c = c_norm * (coeffs_max[i] + 1)

        coeffs_decoded.append(c)
        # Mettre à jour l’index de départ pour le tableau de coefficients suivant
        start += length*8
        # Appliquer la transformation IDWT
        signal_reconstructed = pywt.waverec(coeffs_decoded, 'db4').astype("int"+str(sampwidth))
    with wave.open('irm_to_wave.wav', 'wb') as output_file:
        output_file.setnchannels(n_channels)
        output_file.setsampwidth(sampwidth//8)
        output_file.setframerate(sample_rate)
        output_file.writeframes(signal_reconstructed.tobytes())
    return print("your new file is 'irm_to_wave.wav'")
decompress_irm1()

your new file is 'irm_to_wave.wav'
